In [2]:
from ds_xueqiu import *
df=Xueqiu.hist_price("HSI")
df

In [3]:
from at_watch import *
print get_market_watch()

<table border="1" cellspacing="0" cellpadding="6" rules="rows" frame="void">
  <tbody style="text-align:right;font-size:12.5px">
    <tr>
      <td style="text-align:left;font-weight:bold">估值</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
    </tr>
    <tr>
      <td>全市场</td>
      <td>PE:31.29</td>
      <td>27.25%</td>
      <td>PB:2.34</td>
      <td>24.5%</td>
      <td></td>
    </tr>
    <tr>
      <td>关注</td>
      <td>中证500</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
    </tr>
    <tr>
      <td style="text-align:left;font-weight:bold">股指</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
    </tr>
    <tr>
      <td>上证综指</td>
      <td>沪深300</td>
      <td>创业板指</td>
      <td>深证成指</td>
      <td>恒生指数</td>
      <td>BDI指数</td>
    </tr>
    <tr>
      <td>3076.08</td>
      <td>3792.13</td>
      <td>1681.99</td>
      <td>9802</td>
      <td>28603.41</td>
      <td>690</td>
    </t

In [6]:
'''
华宝油气预估净值=T-2日华宝油气净值×（1+T-1日SPSIOP涨跌幅×0.95）×（1+ T-1日USDCNY涨跌幅）
'''

# 导入基本库
import numpy as np
import pandas as pd

# 导入聚宽库
from jqdata import *

# 导入自定义库
from ds_invest import *
from ds_wall import *
from ds_jisilu import *

def get_discount_162411():

    today=pd.datetime.today().strftime('%Y-%m-%d')

    trade_days=get_trade_days(end_date=today, count=10)
    t3=trade_days[-4].strftime('%Y-%m-%d')
    print 'T-3日',t3
    t2=trade_days[-3].strftime('%Y-%m-%d')
    print 'T-2日',t2
    t1=trade_days[-2].strftime('%Y-%m-%d')
    print 'T-1日',t1

    print ''
    
    loop=True
    loop_count=1
    while loop:
        try:
            # T-2日华宝油气净值
            dt=get_extras( 'acc_net_value', '162411.XSHE', end_date=t2, df=True, count=1)
            print dt
            t2_extras=dt['162411.XSHE'].iloc[0]
            if np.isnan(t2_extras):

                df=get_jsl_data('t0_qdii')
                df=df[df['fund_id']=='162411']
                print dt
                t2_extras=float(df['fund_nav'].iloc[0])
            print 'T-2日净值',t2,t2_extras

            # T-1日SPSIOP涨跌幅
            dt=Invest.hist_price('SPSIOP',start_date=t3,end_date=today)
            t1_spsiop=(dt['close'].iloc[-1]-dt['close'].iloc[-2])/dt['close'].iloc[-2]
            print 'T-1日SPSIOP涨跌幅',t1,'%.2f%%'%(t1_spsiop*100)

            # T-1日USDCNY涨跌幅
            dt=Wall.hist_price('UCY',start_date=t2,end_date=t1)
            t1_usdcny=(dt['close'].iloc[-1]-dt['close'].iloc[-2])/dt['close'].iloc[-2]
            print 'T-1日USDCNY涨跌幅',t1,'%.2f%%'%(t1_usdcny*100)


            dt=get_price('162411.XSHE', end_date=today, frequency='daily', fields=None, skip_paused=False, fq='pre', count=1)
            t_price=dt['close'].iloc[0]
            print '当前价格',today,t_price

            # 预估净值
            t_extras=t2_extras*(1+t1_spsiop*0.95)*(1+t1_usdcny)
            print '预估净值',today,'%.3f'%(t_extras)

            print ''

            # 溢价
            t_premium=(t_price-t_extras)/t_extras*100
            print '溢价：%.2f%%'%(t_premium)

            loop=False
        except Exception as e:
            print e
            if loop_count>5:
                return None
            loop_count+=1
            loop=True

    df=pd.DataFrame(data=[
                    ['#华宝油气','','','','',''],
                    ['#溢价率',' 当前价格','%s日净值'%(t2[-2:]),'%s日估值'%(today[-2:]),'SPSIOP','USDCNY'],
                    ['#%.2f%%'%(t_premium),'%.3f'%(t_price),'%.3f'%(t2_extras),'%.3f'%(t_extras),
                        '%.2f%%'%(t1_spsiop*100),'%.2f%%'%(t1_usdcny*100)]
                    ],columns=range(6)) 

    info1=df.to_html(
            index=False,
            header=False,
            classes='')\
            .replace('<table border="1" class="dataframe">','<table border="1" cellspacing="0" cellpadding="6" rules="rows" frame="void" width="100%">')\
            .replace('<td>#','<td style="text-align:left;font-weight:bold">')\
            .replace('<tbody>','<tbody style="text-align:right;font-size:12.5px">')\
            .replace('None','')

    if np.isnan(t_premium):
        return None

    fix_premium=abs(t_premium)
 
    if fix_premium<=1.5:    
        info=[['#无套利机会','' ]] 
    if fix_premium>1.5 and fix_premium<2.5:  
        if t_premium<0:
            info=[
                ['#防守型','' ],
                ['T日','收盘前，场外申购同等场内份额（按T日净值计算份额)'],
                ['T+1日','买入同等场内份额'],
                ['T+2日','场外申购确认'],
                ['T+6日','赎回资金到账'],
                ['T+7日','赎回资金可用'],
                ] 
        elif t_premium>0:  
            info=[
                ['#溢价套利','' ],
                ['#防守型','' ],
                ['T日','收盘前，场外申购同等场内份额（按T日净值计算份额）'],
                ['T+1日','卖出场内份额'],
                ['T+2日','场外申购确认'],
                ['T+3日','场外转场内'],
                ['T+4日','场外转场内确认'],
                ['T+5日','转入场内的份额可以交易'],
                ] 
    elif fix_premium>=2.5:
        if t_premium<0:
            info=[
                ['#折价套利','' ],
                ['#进攻型','' ],
                ['T日','1、赎回场内份额'],
                ['','2、场内买入同等份额'],
                ['T+2日','场外申购确认'],
                ['T+6日','赎回资金到账'],
                ['T+7日','赎回资金可用'],
                ['#防守型','' ],
                ['T日','收盘前，场外申购同等场内份额（按T日净值计算份额）'],
                ['T+1日','买入同等场内份额'],
                ['T+2日','场外申购确认'],
                ['T+6日','赎回资金到账'],
                ['T+7日','赎回资金可用'],
                ]
        elif t_premium>0:  
            info=[
                ['#溢价套利','' ],
                ['#进攻型','' ],
                ['T日','1、卖出场内份额'],
                ['','2、收盘前，场外申购同等份额（按T日净值计算）'],
                ['T+2日','场外申购确认'],
                ['T+3日','场外转场内'],
                ['T+4日','场外转场内确认'],
                ['T+5日','转入场内的份额可以交易'],
                ['#防守型','' ],
                ['T日','收盘前，场外申购同等场内份额（按T日净值计算）'],
                ['T+1日','卖出场内份额'],
                ['T+2日','场外申购确认'],
                ['T+3日','场外转场内'],
                ['T+4日','场外转场内确认'],
                ['T+5日','转入场内的份额可以交易'],
                ]
    
    df=pd.DataFrame(data=info,columns=range(2)) 
    info2=df.to_html(
        index=False,
        header=False,
        classes='')\
        .replace('<table border="1" class="dataframe">','<table border="1" cellspacing="0" cellpadding="6" rules="rows" frame="void" width="100%">')\
        .replace('<td>#','<td style="text-align:left;font-weight:bold">')\
        .replace('<tbody>','<tbody style="text-align:left;font-size:12.5px">')\
        .replace('None','')
    
    
    #返回信息
    return info1 + info2    
        
        
# T日纳斯达克100指数期货涨跌幅
# dt=Invest.hist_price('USTEC100',start_date=t1,end_date=today)
# print dt
# t_ustec100=(dt['close'].iloc[-1]-dt['close'].iloc[-2])/dt['close'].iloc[-2]
# print 'T日纳斯达克100指数期货涨跌幅',today,'%.2f%%'%(t_ustec100*100)

# T日USDCNY涨跌幅
# dt=Wall.hist_price('UCY',start_date=t1,end_date=today)
# t_usdcny=(dt['close'].iloc[-1]-dt['close'].iloc[-2])/dt['close'].iloc[-2]
# print 'T日USDCNY涨跌幅',today,'%.2f%%'%(t_usdcny*100)

#  T日WTI原油涨跌幅
# dt=Invest.hist_price('WTIUSD',start_date=t1,end_date=today)
# print dt
# t_wti=(dt['close'].iloc[-1]-dt['close'].iloc[-2])/dt['close'].iloc[-2]
# print 'T日WTI原油涨跌幅',today,'%.2f%%'%(t_wti*100)

print get_discount_162411()

T-3日 2019-03-22
T-2日 2019-03-25
T-1日 2019-03-26

            162411.XSHE
2019-03-25          NaN
            162411.XSHE
2019-03-25          NaN
T-2日净值 2019-03-25 0.0
T-1日SPSIOP涨跌幅 2019-03-26 2.14%
T-1日USDCNY涨跌幅 2019-03-26 0.09%
当前价格 2019-03-27 0.51
预估净值 2019-03-27 0.000

溢价：inf%
<table border="1" cellspacing="0" cellpadding="6" rules="rows" frame="void" width="100%">
  <tbody style="text-align:right;font-size:12.5px">
    <tr>
      <td style="text-align:left;font-weight:bold">华宝油气</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
    </tr>
    <tr>
      <td style="text-align:left;font-weight:bold">溢价率</td>
      <td>当前价格</td>
      <td>25日净值</td>
      <td>27日估值</td>
      <td>SPSIOP</td>
      <td>USDCNY</td>
    </tr>
    <tr>
      <td style="text-align:left;font-weight:bold">inf%</td>
      <td>0.510</td>
      <td>0.000</td>
      <td>0.000</td>
      <td>2.14%</td>
      <td>0.09%</td>
    </tr>
  </tbody>
</table><table border="1" cellspacin

In [2]:
from at_discount import *
print get_discount_162411()

T-1日 2019-03-26,T-2日 2019-03-25,T-3日 2019-03-22
T-2日净值 2019-03-25 0.0
T-1日SPSIOP涨跌幅 2019-03-26 2.14%
T-1日USDCNY涨跌幅 2019-03-26 0.09%
当前价格 2019-03-27 0.510
预估净值 2019-03-27 0.000
溢价 inf%
<table border="1" cellspacing="0" cellpadding="6" rules="rows" frame="void" width="100%">
  <tbody style="text-align:right;font-size:12.5px">
    <tr>
      <td style="text-align:left;font-weight:bold">华宝油气</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
    </tr>
    <tr>
      <td style="text-align:left;font-weight:bold">溢价率</td>
      <td>当前价格</td>
      <td>25日净值</td>
      <td>27日估值</td>
      <td>SPSIOP</td>
      <td>USDCNY</td>
    </tr>
    <tr>
      <td style="text-align:left;font-weight:bold">inf%</td>
      <td>0.510</td>
      <td>0.000</td>
      <td>0.000</td>
      <td>2.14%</td>
      <td>0.09%</td>
    </tr>
  </tbody>
</table><table border="1" cellspacing="0" cellpadding="6" rules="rows" frame="void" width="100%">
  <tbody style="text-align:left;fon

In [10]:
# -*- coding: utf-8 -*-

#雪球网站历史行情数据

#返回数据格式

"""

{"stock":{"symbol":"HKHSI"},"success":"true","chartlist":[{"volume":0,"open":28617.0,"high":28617.0,"close":28545.57,"low":27990.45,"chg":-409.54,"percent":-1.41,"turnrate":0.0,"ma5":28647.132,"ma10":28999.612,"ma20":29972.953,"ma30":30201.684,"dif":-573.56,"dea":-414.77,"macd":-317.59,"lot_volume":0,"timestamp":1530547200000,"time":"Tue Jul 03 00:00:00 +0800 2018"},{"volume":0,"open":28546.32,"high":28642.34,"close":28241.67,"low":28141.28,"chg":-303.9,"percent":-1.06,"turnrate":0.0,"ma5":28519.186,"ma10":28876.964,"ma20":29835.138,"ma30":30108.142,"dif":-608.15,"dea":-453.42,"macd":-309.46,"lot_volume":0,"timestamp":1530633600000,"time":"Wed Jul 04 00:00:00 +0800 2018"},{"volume":0,"open":28101.85,"high":28320.9,"close":28182.09,"low":27830.75,"chg":-59.58,"percent":-0.21,"turnrate":0.0,"ma5":28484.352,"ma10":28725.556,"ma20":29689.57,"ma30":30006.4,"dif":-632.93,"dea":-489.12,"macd":-287.61,"lot_volume":0,"timestamp":1530720000000,"time":"Thu Jul 05 00:00:00 +0800 2018"},{"volume":0,"open":28254.37,"high":28554.21,"close":28315.62,"low":27925.33,"chg":133.53,"percent":0.47,"turnrate":0.0,"ma5":28448.012,"ma10":28627.513,"ma20":29542.396,"ma30":29928.066,"dif":-633.86,"dea":-517.11,"macd":-233.49,"lot_volume":0,"timestamp":1530806400000,"time":"Fri Jul 06 00:00:00 +0800 2018"}]}

"""



#导入库
from datetime import timedelta,date,datetime
import pandas as pd
import requests,json,os,re,time

#代码对照
_codes={
    'HSI':'HKHSI', #恒生指数
    'HSCEI':'HKHSCEI', #恒生国企指数   
    'DJI':'.DJI', #恒生国企指数   
    'SPX':'SP500', #标普500
    'NDAQ':'QQQ', #纳斯达克
    }

#日期转换成时间戳
def date_to_timestamp(date):
    return int(time.mktime(time.strptime(date,'%Y-%m-%d')))

#时间戳转换成日期
def timestamp_to_date(timestamp):
    return time.strftime('%Y-%m-%d',time.localtime(timestamp))

class Xueqiu(object):    
    
    #页面头
    @staticmethod
    def __init_xq():
        xueqiu_s=requests.session()
        user_agent='Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 \
(KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36'
        xueqiu_s.headers.update({'User-Agent':user_agent})
        xueqiu_s.headers.update({'X-Requested-With':'XMLHttpRequest'})
        xueqiu_s.get('https://xueqiu.com/')
        return xueqiu_s
    
    @staticmethod
    def hist_price(code,start_date=None,end_date=None,period='D'):    
        
        """
        获取历史行情数据，限制开始、结束时间版本
        code：代码，str
        start_date：开始日期，str，如：'2018-07-01'
        end_date：结束日期，str，如：'2018-07-31'
        period：暂不支持
        """
        # 今天日期
        today=datetime.now().date()
        today_str=today.strftime('%Y-%m-%d')
        
        # 约束开始日期
        if start_date>today_str:
            return None
        
        # 初始化
        xueqiu_s=Xueqiu.__init_xq()
        
        # 代码转换
#         code=_codes[code]
    
        # 约束结束日期
        if end_date is None or end_date>=today_str:
            # 今天日期减一天（与其他数据源保持一致）
            end_date=(today-timedelta(1)).strftime('%Y-%m-%d')

        # url，拼接代码、日期（转换为时间戳，并*1000，以符合雪球时间戳标准）、当前时间
        url='https://xueqiu.com/stock/forchartk/stocklist.json?symbol=%s&period=1day&type=normal'%(code)
        if not start_date is None:
            url='%s&begin=%d'%(url,date_to_timestamp(start_date)*1000)
        if not end_date is None:
            url='%s&end=%s'%(url,date_to_timestamp(end_date)*1000)
        url='%s&_=%d'%(url,int(time.time()*1000))
        print url
        #获取数据
        response=xueqiu_s.get(url,timeout=5)
        # 判断返回标志
        if not response.ok:
            return None
        # 解析数据
        ret_data=response.json()['chartlist']
        # 无数据返回空列表
        if len(ret_data)==0:
            return None
        # 生成数据表
        df=pd.DataFrame(data=ret_data)[['timestamp','open','close','high','low']] 
        # 转换时间戳为标准日期格式
        df['timestamp']=df['timestamp'].apply(lambda x:timestamp_to_date(x/1000))
        # 重置索引
        df=df.set_index('timestamp')
        df.index.name=None
        # 转换格式
        df.index=pd.to_datetime(df.index)
        
        # 返回数据表
        return df
    
    
        
    #行情
    #code:sh000300
    #begin/end: 2016-09-01
    #days:指定最近天数
    @staticmethod
    def price(code,begin='',end='',days=0,timeout=5):        
        xueqiu_s=Xueqiu.__init_xq()

        #code=_codes[code]
        begin=begin.replace('-','')
        end=end.replace('-','')
        if len(begin) > 0 and days > 0: #!!!不能同时指定begin和days!!!
            log.error('begin: %s, days: %d', begin, days)
            return None
        print begin ,end
        #url
        url='https://xueqiu.com/stock/forchartk/stocklist.json?symbol=%s&period=1day&type=normal'%(code)
        if len(begin) > 0:
            url='%s&begin=%d'%(url,int(time.mktime(datetime.strptime(begin,'%Y%m%d').timetuple())*1000))
        if len(end) > 0:
            url='%s&end=%s'%(url,int(time.mktime(datetime.strptime(end,'%Y%m%d').timetuple())*1000))
        if days > 0:
            url='%s&begin=%d'%(url,int(time.mktime((datetime.today()-timedelta(days)).timetuple())*1000))
        url='%s&_=%d'%(url,int(time.time()*1000))
        print url
        #获取网站数据
        r=xueqiu_s.get(url,timeout=timeout)
        if r.ok is False:
            return None
        #解析数据
        ret_data=r.json()
        #有效数据
        stock_data=ret_data['chartlist']
        #数据列表
        data_list=[]
        #日期列表
        day_list=[]
        for item in stock_data:
            #日期格式转换
            try:
                date=datetime.strptime(item['time'],"%a %b %d %H:%M:%S +0800 %Y")
            except:
                date=datetime.strptime(item['time'],"%a %b %d %H:%M:%S +0900 %Y")
            data_list.append([item['open'],item['close'],item['high'],item['low']])
            day_list.append(date)
        df=pd.DataFrame(data=data_list,index=day_list,columns=['open','close','high','low'])   
        return df
    
    
    
Xueqiu.hist_price('sz162411',start_date='2019-03-10',end_date='2019-03-15')   

https://xueqiu.com/stock/forchartk/stocklist.json?symbol=sz162411&period=1day&type=normal&begin=1552147200000&end=1552579200000&_=1553072916528


,open,close,high,low
2017-03-15,0.622,0.624,0.624,0.620
2017-03-16,0.633,0.632,0.634,0.631
2017-03-17,0.630,0.630,0.632,0.629
2017-03-20,0.629,0.626,0.629,0.624
2017-03-21,0.626,0.630,0.631,0.626
2017-03-22,0.619,0.617,0.619,0.615
2017-03-23,0.612,0.616,0.617,0.612
2017-03-24,0.613,0.613,0.614,0.609
2017-03-27,0.611,0.607,0.613,0.607
2017-03-28,0.609,0.610,0.611,0.608


In [21]:
from ds_sina import *

,162411.XSHE
2019-03-15,NaN


In [4]:
# -*- coding: utf-8 -*-

#爬取新浪实时数据

#到入库
import pandas as pd
import numpy as np
import requests
import json
import re
from datetime import timedelta,date,time,datetime

_codes_real=[
    
    #沪深指数（实时价格）
    's_sh000001',#上证指数
    's_sz399001',#深证成指
    's_sh000300',#沪深300

    #香港指数（实时价格）
    'r_HSI',  #恒生指数
    'r_HSCEI',#国企指数
    'r_HSCCI',#红筹指数
    
    #美国指数（实时价格）
    'int_dji',#道琼斯
    'int_nasdaq',#纳斯达克
    'int_sp500',#标普500

    #其它国家指数（实时价格）
    'b_DAX',#德国DAX30
    'b_UKX',#富时100指数
    'b_INDEXCF',#俄罗斯MICEX指数
    'b_CAC',#法国CAC40指数
    'b_SMI',#瑞士市场指数
    'b_FTSEMIB',#富时意大利MIB指数
    'b_MADX',#西班牙MA马德里指数
    'b_OMX',#OMX斯德哥尔摩30指数
    'b_HEX',#OMX赫尔辛基指数
    'b_OSEAX',#挪威OSE全股指数
    'b_ISEQ',#爱尔兰综合指数
    'b_AEX',#荷兰AEX指数
    'b_IBEX',#西班牙IBEX 35指数
    'b_SX5E',#道琼斯欧元区斯托克50指数
    'b_XU100',#土耳其 ISE National-100指数
    'b_NKY',#日经225指数
    'b_HSI',#香港恒生指数
    'b_TWSE',#台湾台北指数
    'b_FSSTI',#富时新加坡海峡时报指数
    'b_KOSPI',#韩国KOSPI指数
    'b_FBMKLCI',#吉隆坡综合股价指数
    'b_SET',#S泰国股票交易指数
    'b_JCI',#印尼雅加达综合指数
    'b_PCOMP',#菲律宾综合股价指数
    'b_KSE100',#巴基斯坦卡拉奇100指数
    'b_VNINDEX',#越南胡志明市股票指数
    'b_CSEALL',#斯里兰卡科伦坡指数
    'b_SASEIDX',#沙特阿拉伯TADAWUL股票综合指数
    'b_INDU',#道琼斯30种工业股票平均价格指数
    'b_CCMP',#纳斯达克综合指数
    'b_SPX',#标准普尔500指数
    'b_SPTSX',#S&P/TSX综合指数
    'b_MEXBOL',#墨西哥BOLSA指数
    'b_IBOV',#巴西BOVESPA股票指数;
    'b_MERVAL',#阿根廷MERVAL指数
    'b_NZSE50FG',#新西兰50指数
    'b_CASE',#埃及CASE 30指数
    'b_JALSH',#FTSE/JSE 南非综合指数

    #商品代码（实时价格）
    'hf_GC',#COMEX黄金
    'hf_OIL',#布伦特原油
    'hf_SI',#COMEX白银
    'hf_GC',#COMEX黄金
    'hf_CL',#NYMEX原油
    'hf_S',#CBOT黄豆
    'hf_C',#玉米
    'hf_TRB',#日本橡胶
    'hf_CAD',#LME铜
    'hf_ZSD',#LME锌
    'hf_AHD',#LME铝
    ]

#商品历史数据代码
_codes_hist={
    "GC",#comex黄金
    "SI",#comex白银
    "XAU",#伦敦金
    "XAG",#伦敦银
    "CL",#NYMEX原油
    "OIL",#布伦特原油
    }

class Sina(object):  

    #从sina获取历史
    @staticmethod
    def hist_price(item,start_date=None,end_date=None):   
        
        # 今天日期
        today_str=datetime.now().date().strftime('%Y-%m-%d')

        # 约束开始日期
        if start_date>today_str:
            return None
            
        # 约束结束日期
        if end_date is None or end_date>today_str:
        # 今天日期
            end_date=today_str
        
        #url链接
        url="http://stock2.finance.sina.com.cn/futures/api/json.php/\
GlobalFuturesService.getGlobalFuturesDailyKLine?symbol=%s"%(item)
        #网络请求数据
        ret=requests.get(url).text 
        #预处理数据
        data=ret.replace('date', '"date"').replace('open', '"open"').replace('high', '"high"')\
.replace('low', '"low"').replace('close', '"close"').replace('volume', '"volume"')  
        #整理数据
        df=pd.DataFrame(json.loads(data))
        print ret,url
        df['date']=pd.to_datetime(df['date'],format='%Y-%m-%d')
        df.set_index('date',inplace=True)
        df.index.name=None
        
        if start_date is None:
            start_date=df.index[0].date().strftime('%Y-%m-%d')
            
        #保存到cvs文件
        return df[(df.index>=start_date) & (df.index<=end_date)].astype('float')  
    
    
    #数据实时价格
    #item_list：代码列表，可一次返回多格代码的数据
    @staticmethod
    def __real_price(item_list,position=1):  
        url_base="http://hq.sinajs.cn/list="
        #list转换为字符串
        item=str(item_list).replace("'",'').replace('[','').replace(']','').replace(' ','')
        #拼接url
        url=url_base+item
        #网络请求数据
        data=requests.get(url).text   
        print data
        #数据list
        data_list=[] 
        temp_list=[]
        #去除多余的字符
        data=re.sub(r'[a-z_=A-Z]',r'',data)
        #分行
        temp_list=data.replace('"','').replace(' ','').split(';\n')
        #遍历列表
        for data in temp_list[0:len(temp_list)-1]: 
            #取position位置的数据（指数=1，商品=0）
            data_list.append(data.split(',')[position])   
        #转换成代码/数据字典    
        return dict(zip(item_list,data_list))


    #商品实时价格
    @staticmethod
    def cmm_real_price(codes): 
        data=Sina.__real_price(codes,0)
        return data

    #指数实时价格
    @staticmethod
    def idx_real_price(codes): 
        data=Sina.__real_price(codes,1)
        return data

    
Sina.hist_price('s_sz162411',start_date='2019-03-15',end_date='2019-03-15')   

[] http://stock2.finance.sina.com.cn/futures/api/json.php/GlobalFuturesService.getGlobalFuturesDailyKLine?symbol=s_sz162411


KeyError: 'date'

In [7]:



#华尔街见闻网站行情数据

#通用模块
import numpy as np
import pandas as pd
import json
import requests
import time  
from datetime import timedelta,date,datetime
from tl import date_to_timestamp,timestamp_to_date



_codes={
    # 港股指数  
    'HSI':'HKG33INDEX',
    # 美股指数      
    'SPX':'SPX500INDEX',
    'DJIA':'US30INDEX',  
    'NDAQ':'NASINDEX',
    # 欧洲 指数  
    'GDAXI':'GER30INDEX', 
    'FTSE':'UK100INDEX',
    'FCHI':'FRA40INDEX',
    'SX5E':'EUSTX50INDEX',
    # 亚洲指数  
    'N225':'JPN225INDEX',
    # 指标
    'VIX':'SP500VIXINDEX',
    'BDI':'BDIINDEX',

    'UDI':u'USDOLLARINDEX',
    'VIX':u'SP500VIXINDEX',
    'BDI':u'BDIINDEX',
    'C10Y':u'CHINA10YEAR',
    'C5Y':u'CHINA5YEAR',
    'U10Y':u'US10YEAR',
    'U5Y':u'US5YEAR',
    'UCH':'USDCNH',
    'USCL':'USCL',
    'USTEC100F':'USTEC100F',#纳斯达克100指数期货
    }


class Wall(object):
    """
    获取华尔街见闻网站历史行情日线数据

    url格式：
    https://api-ddc.wallstreetcn.com/market/kline?prod_code=CHINA10YEAR&candle_period=8&data_count=1&end_time=1522252800&
       fields=time_stamp,open_px,close_px,high_px,low_px
    返回的数据格式：
    {"code":200,"data":{"candle":{"CHINA10YEAR":[[1522191600,3.773,3.758,3.773,3.747]],
        "fields":["time_stamp","open_px","close_px","high_px","low_px"]}}}

    返回的数据不含当天的数据
    prod_code：代码
    candle_period：数据级别，8代表日线数据
    data_count：每次返回的数据量，网站限制每次最多返回1000条数据
    end_time：截至时间，网站返回的日期格式为时间戳，如1522191600
    """
    
    @staticmethod
    def hist_price(code,start_date=None,end_date=None,period='D'):  
        """
        获取历史行情数据，限制开始、结束时间版本
        code：代码，str
        start_date：开始日期，str，如：2018-07-01
        end_date：结束日期，str，如：2018-07-31
        period：数据线级别，str，分别为：'D'、'W'、'M'（日、周、月）
        """
                
        #今日日期
        today=datetime.now().date()
        today_str=today.strftime('%Y-%m-%d')
        
        # 约束开始日期
        if start_date>today_str:
            return None
        
        # 转换代码
       # code=_codes[code]  
        
        # 约束开始日期
        if not start_date is None:
            # 转换日期为timestamp
            start_date=date_to_timestamp(start_date)
            
        # 结束日期
        if end_date is None:
            # 今日日期
            end_date=today_str
        else:
            # 日期+1天(否则取不到end_date当天的值)
            end_date=(datetime.strptime(end_date,'%Y-%m-%d')+timedelta(1)).strftime('%Y-%m-%d')         
        # 转换日期为timestamp
        end_date=date_to_timestamp(end_date)    
        
        # 每次取数据时的日期值
        end_time=end_date          
        # 转换数据线级别（# '1m':1,'5m':2,'15m':3,'30m':4,'1h':5,'4h':7,）
        period={'D':8,'W':10,'M':11}[period]
        # 默认每次获取的数据量（不能超过1000）
        data_count=1000

        df=pd.DataFrame()
        data_list=[]
        
        while True:

            # 生成url 
            url='https://api-ddc.wallstreetcn.com/market/kline?prod_code={prod_code}\
&candle_period={candle_period}&data_count={data_count}&end_time={end_time}\
&fields=time_stamp%2Copen_px%2Cclose_px%2Chigh_px%2Clow_px'.format(prod_code=code,
                   candle_period=period,data_count=data_count,end_time=end_time)

            # 请求数据
            response=requests.get(url).text
            print url
            # 获取数据列表
            data=json.loads(response).get("data").get("candle").get(code)
           
            # 无数据退出
            if len(data)==0:
                break
                
            # 反转数据    
            data.reverse()

            # 查找指定日期范围内的数据
            b=False
            for item in data:
                if item[0]<=end_date:
                    if item[0]<start_date:
                        b=True
                        break
                    data_list.append(item)
            # 找到指定日期范围内的所有数据，退出        
            if b:
                break

            # 取得下一次的时间戳
            end_time=data[-1][0]

            # 延时
            time.sleep(0.0)
        

        # 无数据
        if len(data_list)==0:
            return None
        
        #组织数据
        df=pd.DataFrame(data_list,columns=['date','open','close','high','low'])
        
        # 时间戳转换为日期    
        df['date']=df['date'].apply(lambda x:timestamp_to_date(x))
        
        #去除重复数据（不知为何返回的数据有重复,必须按照日期删除重复）
        df=df.drop_duplicates(['date'])  
        
        # 重置索引
        df=df.set_index('date')
        df.index.name=None

        # 转换索引为日期格式
        df.index=pd.to_datetime(df.index,format='%Y-%m-%d')
        
        # 保留2位小数位
#         df=np.round(df,2)
        
        #返回数据（升序）
        return df.sort_index()   
     
Wall.hist_price('wti-usd-historical-data',start_date='2019-03-01',end_date='2019-03-14')


https://api-ddc.wallstreetcn.com/market/kline?prod_code=wti-usd-historical-data&candle_period=8&data_count=1000&end_time=1552579200&fields=time_stamp%2Copen_px%2Cclose_px%2Chigh_px%2Clow_px


AttributeError: 'NoneType' object has no attribute 'get'

In [14]:



"""
investing网站历史行情数据
网站：
https://tvc-cncdn.investing.com'
网址：
https://tvc4.forexpros.com/a5cb6ed3c0b48d5dc77f0991b1717479/1531986161/6/6/28/history?symbol=29227&resolution=D&
from=1527782400&to=1528387200
返回数据格式：
{u'c': [3.6480000019073, 3.6670000553131, 3.683000087738, 3.6930000782013, 3.6930000782013], u'h': [3.6659998893738,
3.6800000667572, 3.6930000782013, 3.7000000476837, 3.7070000171661], u'l': [3.6480000019073, 3.6500000953674, 
3.6730000972748, 3.6779999732971, 3.6800000667572], u'o': [3.6659998893738, 3.6800000667572, 3.6800000667572, 
3.6979999542236, 3.6970000267029], u'vo': [u'n/a', u'n/a', u'n/a', u'n/a', u'n/a'], u's': u'ok', u't': [1527811200, 
1528070400, 1528156800, 1528243200, 1528329600], u'v': [u'n/a', u'n/a', u'n/a', u'n/a', u'n/a']}
无数据返回：
{u'nextTime': 1527724800, u's': u'no_data'}
通过判断's'状态，获悉有无数据
"""


#导入库
import pandas as pd
import numpy as np
import time
import requests
import json
from datetime import timedelta,date,datetime

#代码对照
_codes={
    'HSI':'HK50', #恒生指数
    'HSCEI':'HKHSCEI', #恒生国企指数   
    'DJI':'DJI', #道琼斯指数
    'SPX':'US500', #标普500指数
    'NDAQ':'IXIC', #纳斯达克综合指数
    'GDAXI':'DE30', #德国DAX30
    'FTSE':'UK100', #英国富时100
    'FCHI':'F40', #法国CAC40
    'SX5E':'STOXX50', #欧洲斯托克50
    'N225':'JP225', #日经225
    'C10Y':'CN10YT',#中国10年期国债
    'SPSIOP':'SPSIOPTR'
    }

#日期转换成时间戳
def date_to_timestamp(date):
    return int(time.mktime(time.strptime(date,'%Y-%m-%d')))

#时间戳转换成日期
def timestamp_to_date(timestamp):
    return time.strftime('%Y-%m-%d',time.localtime(timestamp))

class Invest(object):    
    
    #页面头
    @staticmethod
    def hist_price(code,start_date=None,end_date=None,period='D'): 
        # 代码转换
        #code=_codes[code]
        
        # 参数约束
        if start_date is None:
            start_date='1900-01-01'
        if end_date is None:
            end_date=datetime.now().date().strftime('%Y-%m-%d')
        if not period in ['D','W','M']:    
            period='D'
        print    start_date,end_date 
        # 创建session
        session=requests.session()
        user_agent='Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 \
    (KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36'
        session.headers.update({'User-Agent':user_agent})
        session.get('https://tvc-cncdn.investing.com')
        
        # url
        url='https://tvc4.forexpros.com/a5cb6ed3c0b48d5dc77f0991b1717479/1531986161/6/6/28/history?\
symbol=%s&resolution=%s&from=%s&to=%s'%(code,period,date_to_timestamp(start_date),date_to_timestamp(end_date))
        print url
        # 请求数据
        response=session.get(url)
        if not response.ok:
            return None
        
        # 解析数据
        data=json.loads(response.text)
        
        # 判断是否有数据
        if not data.get('s') =='ok':
            return None
        
        # 生成数据表
        df=pd.DataFrame(data={'close':data.get('c'),'high':data.get('h'),'low':data.get('l'),'open':data.get('o')})
        # 转换日期格式
        df.index=pd.to_datetime(map(lambda x:timestamp_to_date(x),data.get('t')),format='%Y-%m-%d')
        # 保留2位小数位
#         df=np.round(df,2)
        
        # 返回数据
        return df
    

session=requests.session()
user_agent='Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 \
(KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36'
session.headers.update({'User-Agent':user_agent})
session.get('https://tvc-cncdn.investing.com')

# url
url='https://sbcharts.investing.com/charts_xml/c4caa3c525726eb96b6ee67ddfa57896_1day.json'
url='https://sbcharts.investing.com/charts_xml/a8fb316cc08a30c3c16be55d2e7baeda_1day.json'
print url
# 请求数据
response=session.get(url)

# 解析数据
json.loads(response.text)

https://sbcharts.investing.com/charts_xml/a8fb316cc08a30c3c16be55d2e7baeda_1day.json


{u'_comment': u'pair_id: 2111, interval: 300, period: 1day',
 u'attr': {u'decimals': 4,
  u'exchange_id': 1001,
  u'interval': u'300',
  u'last_close_value': 6.7129,
  u'last_update_datetime': None,
  u'last_value': 6.7138,
  u'pair_id': 2111,
  u'server_time': 1552817725,
  u'symbol': u'USD/CNY - US Dollar Chinese Yuan'},
 u'candles': [[1552576500000, 6.723],
  [1552613400000, 6.7222],
  [1552613700000, 6.7263],
  [1552614000000, 6.7248],
  [1552614300000, 6.7249],
  [1552614600000, 6.7243],
  [1552614900000, 6.724],
  [1552615200000, 6.7247],
  [1552615500000, 6.7249],
  [1552615800000, 6.7275],
  [1552616100000, 6.7296],
  [1552616400000, 6.7288],
  [1552616700000, 6.7285],
  [1552617000000, 6.727],
  [1552617300000, 6.7263],
  [1552617600000, 6.7263],
  [1552617900000, 6.726],
  [1552618200000, 6.7251],
  [1552618500000, 6.7189],
  [1552618800000, 6.718],
  [1552619100000, 6.7172],
  [1552619400000, 6.7191],
  [1552619700000, 6.7208],
  [1552620000000, 6.7197],
  [1552620300000, 6.

In [ ]:
# 聚源数据、交易日
# from ds import *


In [1]:
import numpy as np
import pandas as pd
import math
import os



# 聚源数据、交易日
from ds import dsIdx,dsMcr,dsFnd,dsCmm,dsPlt,dsIdu,dsStk,dsData

from tl import *

In [10]:
import time 
from datetime import timedelta,date,datetime

now = datetime.now()
month = (now.month - 1) - (now.month - 1) % 3 + 1


# 上季最后一天日期（本月第一天日期减去一天）
start_date =datetime(now.year,month,1)-timedelta(days=1)
# 本季最后一天日期
if month>9:
    end_date=datetime(now.year,12,31)
else:  
    end_date = datetime(now.year,month+3,1)-timedelta(days=1)

print start_date,end_date

2018-09-30 00:00:00 2018-12-31 00:00:00


In [5]:
get_divid(get_index_stocks('000300.XSHG'),'2018-09-01')

(2.7790555985536525, 307122.4883000002)

In [3]:
get_divid('600900.XSHG','2018-09-01')

['600900.XSHG']


(4.298356634247692, 3480.3999)

In [6]:
from pf_idx import Index
index=Index('csv','Data/')

In [ ]:
def get_divid(self,code,end_date):
    """
    指定日期股息率,jy版本
    code：股票代码,list or str
    end_date：截至日期
    返回：股息率、市值
    """
    # 判断代码是str还是list
    if not type(code) is list:
        code=[code]

    #聚宽代码转换为证券代码
    SecuCodes=code_jq_scu(code) 
    #证券代码转换为jy内部代码    
    InnerCodes=code_scu_jy(SecuCodes)

    #获取所有成份股派息总额
    #数据表    
    df=pd.DataFrame()
    #因jy每次最多返回3000条数据，所以要多次查询
    #偏移值
    offset=0    
    while True:
        #查询语句
        q=query(
            #内部代码
            jy.LC_Dividend.InnerCode,
            #派息日期
            jy.LC_Dividend.ToAccountDate,
            #派息总额
            jy.LC_Dividend.TotalCashDiviComRMB,
        ).filter(
            # 已分红
            jy.LC_Dividend.IfDividend==1,
            # 获取指定日期前所有分红
            jy.LC_Dividend.ToAccountDate<=end_date,
            jy.LC_Dividend.InnerCode.in_(InnerCodes)
        #偏移         
        ).offset(offset)
        #查询    
        temp_df=jy.run_query(q)  
        if len(temp_df)==0:
            break
        #追加数据
        df=df.append(temp_df)
        #偏移值每次递增3000    
        offset+=3000

    if len(df)==0:
        div=float('NaN')
    else:
        # 生成排序字段  
        df['sort']=df['InnerCode'].astype('str')+df['ToAccountDate'].astype('str').str[0:10]
        df=df.sort('sort') 
        # 只保留最后一次派息数据
        df=df.drop_duplicates('InnerCode',take_last=True) #keep='last'
        # 返回合计的派息数
        div=df['TotalCashDiviComRMB'].sum()

    #获取指数总市值 
    q=query(
        #市值
        valuation.market_cap
    ).filter(
        valuation.code.in_(code)
    )
    #获取各成份股市值(亿元)
    df=get_fundamentals(q,end_date)
    #返回合计的成份股总市值（亿元）
    cap=df['market_cap'].sum()

    try:
        #返回股息率
        return div/cap/100000000*100.0,cap
    except:
        return float('NaN'),float('NaN')